## Creating Medallion Architecture

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS enterprise_modernization.bronze;
CREATE SCHEMA IF NOT EXISTS enterprise_modernization.silver;
CREATE SCHEMA IF NOT EXISTS enterprise_modernization.gold;


## Bronze to Silver transformation

### SAP

In [0]:
sap_bronze = spark.table("enterprise_modernization.bronze.bronze_sap")
sap_silver = (
    sap_bronze
    .dropDuplicates(["vehicle_id"])
    .filter("price > 0 and manufacturer IS NOT NULL and model IS NOT NULL")
    .withColumn("year_of_manufacture", sap_bronze["year_of_manufacture"].cast("int"))
    .withColumn("engine_size", sap_bronze["engine_size"].cast("double"))
)
sap_silver.write.mode("overwrite").saveAsTable("enterprise_modernization.silver.silver_sap")

### CRM

In [0]:
crm_bronze = spark.table("enterprise_modernization.bronze.bronze_crm")
crm_silver = (
    crm_bronze
    .dropDuplicates(["order_id"])
    .filter("sales > 0 and customer_id IS NOT NULL and vehicle_id IS NOT NULL")
    .withColumn("order_date", crm_bronze["order_date"].cast("date"))
)
crm_silver.write.mode("overwrite").saveAsTable("enterprise_modernization.silver.silver_crm")


### Fleet

In [0]:
fleet_bronze = spark.table("enterprise_modernization.bronze.bronze_fleet")
fleet_silver = (
    fleet_bronze
    .dropDuplicates(["device_id", "timestamp"])
    .filter("battery_level >= 0 and fault_detected IN (0, 1)")
    .withColumn("timestamp", fleet_bronze["timestamp"].cast("timestamp"))
)
fleet_silver.write.mode("overwrite").saveAsTable("enterprise_modernization.silver.silver_fleet")
